<h1>Memory Usage Tools</h1>
<h2>By Thomas Hilton Johnson III and Brian Vanderwende</h2>

<p>The Notebook displayed here is for the purpose of providing an available reference for NCAR HPC resource users to understand their memory usage over time. There is also code here in bash for the purpose of of providing reference as to how to use tools to evaluate memory usage among other satistics when running jobs. This is to give users a better understanding of the tools available to help said users be as efficient and efective as possible when running jobs.</p>

<h3>Set Up</h3>
<p>Make sure that you have the appropriate modules loaded for this tutorial. The next set of commands will ensure such.</p>

In [1]:
module purge
ml ncarenv
ml intel
ml ncarcompilers
module load intel impi peak_memusage
module li
module load arm-reports


Currently Loaded Modules:
  1) ncarenv/1.2    3) ncarcompilers/0.4.1   5) peak_memusage/2.0.1
  2) intel/17.0.1   4) impi/2017.1.132

 

>

<p>Next, check to make sure that the correct modules have been loaded.</p>

In [2]:
module list


Currently Loaded Modules:
  1) ncarenv/1.2    3) ncarcompilers/0.4.1   5) peak_memusage/2.0.1
  2) intel/17.0.1   4) impi/2017.1.132       6) arm-reports/19.0.4

 

>

<p>Next, verify that there is file called <code>contstructMultiDimensionalItem.f90</code> present in current directory.</p>

In [3]:
ls

casper_execute_fortran_peakmemusage.sh
casper_fortran_script_for_execution.sh
cheyenne_execute_fortran_ddt.sh
cheyenne_execute_fortran_gprof.sh
cheyenne_execute_fortran_peakmemusage.sh
cheyenne_fortran_script_for_execution.sh
contstructMultiDimensionalItem.f90
core-cheyenne2-32874-47142-11
dav_jupyter_env
dav_jupyter_ncar_env
execution_code_fortran
execution_code_fortran_1p_1n_1t_2019-07-16_16-10.html
execution_code_fortran_1p_1n_1t_2019-07-16_16-10.txt
execution_code_fortran_pile
experimentalmultidimensionalitem.f90
fortran_casper_job_output
fortran_cheyenne_peakmemusage.o7191473
fortran_cheyenne_test.o7178629
Memory_Usage_Tools_Casper.ipynb


<p>If the file fortran file is present, feel free to proceed to the next section.</p>
<h3>Starting Within an Interactive Job</h3>
<p>The before mentioned tools to be taught that are to be taught through this tutorial need a job to be ran in order to be used. One way to run jobs is to send scripts with all the necessary details to be queued and ran by the machine. Another way to do such is to run an interactive job, in which you are hands-on in constructing and defining the job.</p>

<p>To start using the fortran code, first we must compile it and generate an executable. If using the Intel compiler, use ifort. If using grfortran, use gfortran. The next part is using ifort with flags and arguments to compile the fortran code.</p>

In [2]:
ifort -g -o execution_code_fortran_pile contstructMultiDimensionalItem.f90 

<p>An executable should have been generated. Now, for the next part. We will run the executable with the correct arguments. The first tool we will use is Peak Memory Usage. To Use this tool it is necessary to first load the Peak Memory Usage Module using <code>module load intel impi peak_memusage</code>. When running the Fortran executable, we must also add the <code>peak_memusage.exe</code> executable before the Fortran executable. There are also four integer arguments that must be placed to determine the size of each array in the multidimensional array structure. The arguments are added after the Fortran executable. The following section of code is the correct means to set up said code execution.</p>

In [12]:
peak_memusage.exe ./execution_code_fortran_pile 350 350 350

Running: ./execution_code_fortran_pile 350 350 350; sleep 1  - Please wait...
 Please put the dimensions of the three dimensional array:
 Input the first dimension:
         350
 Input the second dimension:
         350
 Input the third dimension:
         350
 New array is formed.
 Iteration:           1
 New array is formed.
 Iteration:           2
 New array is formed.
 Iteration:           3
 New array is formed.
 Iteration:           4
 New array is formed.
 Iteration:           5
 New array is formed.
 Iteration:           6
 New array is formed.
 Iteration:           7
 New array is formed.
 Iteration:           8
 New array is formed.
 Iteration:           9
 New array is formed.
 Iteration:          10
 New array is formed.
 Iteration:          11
 New array is formed.
 Iteration:          12
 New array is formed.
 Iteration:          13
 New array is formed.
 Iteration:          14
 New array is formed.
 Iteration:          15
 New array is formed.
 Iteration:          16
 Ne

<p>The previous output is the result of running the executable compiled from the Fortran code. Do note that the time it takes to process said output is dependent on the three integer inputs that are give. The larger the integer inputs, the more time that the processing will take in comparison to smaller integer inputs. Also, the larger the integer inputs, the more memeory that will be required to run the executable. After the program's output is displayed, the output of the Peak Memory Usage tool will be displayed showing the memory that was required to run the program.</p>

<p>The next tool to be displayed is an Arm Performance Report that is made possible by loading the <code>arm-reports</code> module. This module enables users to generate an HTML and text file, displaying identical information, as to various statisics regarding the executable code that has been ran.</p>

<p>Next we will generate a performance report. To do so, add perf-report before the exectable and the executable's arguments. This will ensure that the job is ran and will output HTML and text files to store information regarding the perfromanc of the code that has been ran.</p>

In [6]:
perf-report ./execution_code_fortran_pile 350 350 350

Configuring environment for Forge...
  TMPDIR=/glade/scratch/thomasjo
  CONFIG=/glade/u/home/thomasjo/.allinea/dav
  MPILIB=impi-2017.1.132.so

 Please put the dimensions of the three dimensional array:
 Input the first dimension:
         350
 Input the second dimension:
         350
 Input the third dimension:
         350
 New array is formed.
 Iteration:           1
 New array is formed.
 Iteration:           2
 New array is formed.
 Iteration:           3
 New array is formed.
 Iteration:           4
 New array is formed.
 Iteration:           5
 New array is formed.
 Iteration:           6
 New array is formed.
 Iteration:           7
 New array is formed.
 Iteration:           8
 New array is formed.
 Iteration:           9
 New array is formed.
 Iteration:          10
 New array is formed.
 Iteration:          11
 New array is formed.
 Iteration:          12
 New array is formed.
 Iteration:          13
 New array is formed.
 Iteration:          14
 New array is formed.
 Iterat

<p>The output of the perfromance report will not be displayed merely as output. Instead, two files will be created within the current directory. One of the files is a text file and the other file is an HTML file. The files contain similar content which includes statistics concerning the memory, energy, CPU usage, and Input/Output data. The purpose fo this data is to provide the user with information that is relevant to observing the the charactersistics of the code when ran.</p>

<p>For more information, please refer to the module help information, reference documentation on the software module being utilized, or contact the CISL Consulting Group.</p>